<a href="https://colab.research.google.com/github/steffenmodest/Python_Examples/blob/Protocol/M_Net_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os.path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

rs = 23  # fester Random Seed

In [37]:
# from google.colab import drive
# drive.mount('/content/drive')

Load Data

In [3]:
from tensorflow.keras.datasets import fashion_mnist, mnist

# Fashion MNIST Daten laden, wir wollen nur die Trainingsdaten und verwerfen die Testdaten
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


Own Funs

In [14]:
def ifelse(a):
  if a == 0:
    return 1
  else:
    return a


def rms_norm(v_x):
  n_rms = np.sqrt(np.sum(v_x ** 2)/(len(v_x) - 1))
  return v_x/n_rms


def prot_row(d_results):
    """
    make a new row in protokoll

    Parameters
    ----------
    outcome : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    """
    
    prot_file = 'M-Net_Protocol.xlsx'
    # prot_file = '/content/sample_data/M-Net_Protocol.xlsx'
    if os.path.isfile(prot_file):
        df_prot = pd.read_excel (prot_file)
        df_prot = df_prot.append(d_results, 
                          ignore_index=True)
    else:
        # df_prot = pd.DataFrame.from_dict(d_results)
        df_prot = pd.DataFrame([results])
        # pf_prot = pd.DataFrame.from_records([d_results], index='TIMESTAMP')
        # df_prot = pd.DataFrame(columns=('TIMESTAMP', 
        #                                 'NUMBER_OF_HIDDEN', 
        #                                 'ACCURACY_SCORE', 
        #                                 'MAE_1ST_FL',
        #                                 'MSE_1ST_FL',
        #                                 'R2_1ST_FL',
        #                                 'MAE_1ST_BA',
        #                                 'MSE_1ST_BA',
        #                                 'R2_1ST_BA',
        #                                 'MAE_2ND_FL',
        #                                 'MSE_2ND_FL',
        #                                 'R2_2ND_FL',
        #                                 'MAE_2ND_BA',
        #                                 'MSE_2ND_BA',
        #                                 'R2_2ND_BA',
        #                                 'MSG'))
        
    # s_outcome = outcome.partition('<msg>')[2]
    # s_outcome = s_outcome.partition('</msg>')[0]
    # s_outcome = s_outcome.partition('Set ')[2]
    # s_set = s_outcome.partition(' ')[0]
    # s_msg = s_outcome.partition(' ')[2]
    
    # s_outcome = s_outcome.strip(',()')
    # l_outcome = s_outcome.split(',')
    # df_prot = df_prot.append({'TIMESTAMP':datetime.now(),
    #                           'RESULT': result,
    #                           'SAS_SET':int(s_set),
    #                           'MSG':s_msg,
    #                           'COUNT':n_len}, ignore_index=True)
    
    # df_prot = df_prot.append(d_results, 
    #                          ignore_index=True)
    df_prot.to_excel(prot_file, index=False)
        
    return None

Params and Results

In [6]:
params = {'N_HIDDEN': 67}
results = {'TIMESTAMP': datetime.now(),
           'NUMBER_OF_HIDDEN': params['N_HIDDEN'],
          'ACCURACY_SCORE': 0, 
          'MAE_1ST_FL': 0,
          'MSE_1ST_FL': 0,
          'R2_1ST_FL': 0,
          'MAE_1ST_BA': 0,
          'MSE_1ST_BA': 0,
          'R2_1ST_BA': 0,
          'MAE_2ND_FL': 0,
          'MSE_2ND_FL': 0,
          'R2_2ND_FL': 0,
          'MAE_2ND_BA': 0,
          'MSE_2ND_BA': 0,
          'R2_2ND_BA': 0,
          'MAE_TEST_BA': 0,
          'MSE_TEST_BA': 0,
          'R2_TEST_BA': 0,
          'DICT_REPORT': '',
          'MSG':'M-Net on python'}

In [15]:
results

{'TIMESTAMP': datetime.datetime(2021, 3, 8, 19, 10, 3, 963081),
 'NUMBER_OF_HIDDEN': 67,
 'ACCURACY_SCORE': 0,
 'MAE_1ST_FL': 0,
 'MSE_1ST_FL': 0,
 'R2_1ST_FL': 0,
 'MAE_1ST_BA': 0,
 'MSE_1ST_BA': 0,
 'R2_1ST_BA': 0,
 'MAE_2ND_FL': 0,
 'MSE_2ND_FL': 0,
 'R2_2ND_FL': 0,
 'MAE_2ND_BA': 0,
 'MSE_2ND_BA': 0,
 'R2_2ND_BA': 0,
 'MAE_TEST_BA': 0,
 'MSE_TEST_BA': 0,
 'R2_TEST_BA': 0,
 'DICT_REPORT': '',
 'MSG': 'M-Net on python'}

Preprocess data

In [16]:
prot_row(results)

NEW Norm

In [42]:
X = X_train.reshape(len(X_train), -1).astype('int')
X = pd.DataFrame(X)
X = X.applymap(ifelse)
X = X.apply(rms_norm, axis=1)
X = np.array(X)

In [43]:
X

array([[0.01148061, 0.01148061, 0.01148061, ..., 0.01148061, 0.01148061,
        0.01148061],
       [0.01076957, 0.01076957, 0.01076957, ..., 0.01076957, 0.01076957,
        0.01076957],
       [0.01425251, 0.01425251, 0.01425251, ..., 0.01425251, 0.01425251,
        0.01425251],
       ...,
       [0.01299969, 0.01299969, 0.01299969, ..., 0.01299969, 0.01299969,
        0.01299969],
       [0.01400668, 0.01400668, 0.01400668, ..., 0.01400668, 0.01400668,
        0.01400668],
       [0.01375564, 0.01375564, 0.01375564, ..., 0.01375564, 0.01375564,
        0.01375564]])

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
# X = X_train.reshape(len(X_train), -1).astype('float64')

# # Dimensionen um den Mittelpunkt zentrieren
# preproccessing = StandardScaler()
# X = preproccessing.fit_transform(X)
# X = np.array(X)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_train.shape, np.mean(X_train), np.std(X_train)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

Originaldaten:
Shape: (60000, 28, 28), Mean: 33.318421, STD: 78.567490
Vorbereitete Daten:
Shape: (60000, 784), Mean: 0.396041, STD: 0.917538


Create Hidden Activations

In [45]:
H = np.random.rand(60000, params['N_HIDDEN'])
print(H.shape)
print(H)

(60000, 67)
[[0.93106053 0.35350364 0.82112552 ... 0.10788651 0.44717202 0.97697815]
 [0.5700004  0.00530488 0.14295874 ... 0.00711247 0.12334503 0.19359868]
 [0.85193153 0.93852779 0.88156134 ... 0.75396906 0.45329248 0.43419261]
 ...
 [0.09051823 0.92466098 0.08703636 ... 0.49155371 0.05162382 0.17527535]
 [0.49443382 0.38094372 0.96310446 ... 0.25202613 0.73253873 0.96900983]
 [0.60784893 0.3153874  0.12169218 ... 0.27662878 0.25232604 0.90525516]]


1st Forward Learn

In [46]:
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(H, X)
print(reg.score(H, X))
X_pred = reg.predict(H)

results['MAE_1ST_FL'] = round(mean_absolute_error(X_pred, X), 3)
results['MSE_1ST_FL'] = round(mean_squared_error(X_pred, X), 3)
results['R2_1ST_FL'] = round(r2_score(X_pred, X, multioutput="variance_weighted"), 3)

print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.001121670704077129
MAE 0.4374432950974862
MSE 0.5932398578296855
R2 -919.9263273585073
R2 vw -889.5272515945301
R2 ua -919.9263273585073
(784, 67)
(784,)


1st Back Activations

In [47]:
# # X = X_train.reshape(len(X_train), -1).astype('int')
# W = pd.DataFrame(W)
# # X = X.applymap(ifelse)
# W = W.apply(rms_norm)
# W = np.array(X)

In [48]:
XT = X.transpose()
print(f'shape of X_train {X.shape} and X_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)

results['MAE_1ST_BA'] = round(mean_absolute_error(XT_pred, XT), 3)
results['MSE_1ST_BA'] = round(mean_squared_error(XT_pred, XT), 3)
results['R2_1ST_BA'] = round(r2_score(XT_pred, XT, multioutput="variance_weighted"), 3)

print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

shape of X_train (60000, 784) and X_train transposed (784, 60000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.6601114806662423
MAE 0.3924082327676328
MSE 0.2850368843069982
R2 0.4689840532704006
R2 vw 0.4851043660220658
R2 ua 0.4689840532704006
(60000, 67)
(60000,)


Hidden Norm

In [49]:
# X = H_new_train.reshape(len(H_new_train), -1).astype('int')
H_new = pd.DataFrame(H_new)
# H_new = H_new.applymap(ifelse)
H_new = H_new.apply(rms_norm, axis=1)
H_new = np.array(H_new)

2nd Forward learn - result are Coherent Weights

In [50]:
reg = LinearRegression().fit(H_new, X)
print(reg.score(H_new, X))
X_pred = reg.predict(H_new)

results['MAE_2ND_FL'] = round(mean_absolute_error(X_pred, X), 3)
results['MSE_2ND_FL'] = round(mean_squared_error(X_pred, X), 3)
results['R2_2ND_FL'] = round(r2_score(X_pred, X, multioutput="variance_weighted"), 3)

print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.8095863386991629
MAE 0.18369300119718532
MSE 0.1130878206543544
R2 -9.412989760843127
R2 vw 0.7648012914758515
R2 ua -9.412989760843127
(784, 67)
(784,)


2nd Back Activation - final representation of X_train in Hidden Activations

In [51]:
reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)

results['MAE_2ND_BA'] = round(mean_absolute_error(XT_pred, XT), 3)
results['MSE_2ND_BA'] = round(mean_squared_error(XT_pred, XT), 3)
results['R2_2ND_BA'] = round(r2_score(XT_pred, XT, multioutput="variance_weighted"), 3)

print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.8559070336167399
MAE 0.23287473890473312
MSE 0.12083906296377891
R2 vw 0.8316488114668505
R2 ua 0.8271646566011893
(60000, 67)
(60000,)


In [52]:
# X = H_new_train.reshape(len(H_new_train), -1).astype('int')
H_new = pd.DataFrame(H_new)
# H_new = H_new.applymap(ifelse)
H_new = H_new.apply(rms_norm, axis=1)
H_new = np.array(H_new)

KNN test with Hidden Activations

In [53]:
X = X_test.reshape(len(X_test), -1).astype('int')
X = pd.DataFrame(X)
X = X.applymap(ifelse)
X = X.apply(rms_norm, axis=1)
X = np.array(X)

In [54]:
# X = [[0], [1], [2], [3]]
# y = [0, 0, 1, 1]

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
# X_p_test = X_test.reshape(len(X_test), -1).astype('float64')

# # Dimensionen um den Mittelpunkt zentrieren
# preproccessing = StandardScaler()
# X_p_test = preproccessing.fit_transform(X_p_test)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_test.shape, np.mean(X_test), np.std(X_test)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5, n_jobs=-1, weights='distance')
neigh.fit(H_new, y_train)

# print(neigh.predict([[1.1]]))

# print(neigh.predict_proba([[0.9]]))

Originaldaten:
Shape: (10000, 28, 28), Mean: 33.791224, STD: 79.172463
Vorbereitete Daten:
Shape: (10000, 784), Mean: 0.398067, STD: 0.916661


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='distance')

Back Activation of Test Data

In [55]:
XT = X.transpose()
print(f'shape of X_p_train {X.shape} and X_p_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)

results['MAE_TEST_BA'] = round(mean_absolute_error(XT_pred, XT), 3)
results['MSE_TEST_BA'] = round(mean_squared_error(XT_pred, XT), 3)
results['R2_TEST_BA'] = round(r2_score(XT_pred, XT, multioutput="variance_weighted"), 3)

print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_test = reg.coef_
print(H_test.shape)
HI_test = reg.intercept_
print(HI_test.shape)

shape of X_p_train (10000, 784) and X_p_train transposed (784, 10000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.860421886325071
MAE 0.23010993721107312
MSE 0.11682649972076616
R2 0.8340663278761236
R2 vw 0.837779447625306
R2 ua 0.8340663278761236
(10000, 67)
(10000,)


In [56]:
# X = H_new_train.reshape(len(H_new_train), -1).astype('int')
H_test = pd.DataFrame(H_test)
# H_new = H_new.applymap(ifelse)
H_test = H_test.apply(rms_norm, axis=1)
H_test = np.array(H_test)

In [57]:
# Predicting the Test set results
y_pred = neigh.predict(H_test)

In [58]:
# y_prob = neigh.predict_proba(H_test)

In [59]:
# print(y_prob[0])

In [60]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 974    0    0    0    0    1    4    1    0    0]
 [   0 1128    3    0    3    0    1    0    0    0]
 [  10    2  994    3    0    0    0   12   10    1]
 [   3    0    8  964    1   10    0    7   13    4]
 [   2    4    1    0  941    0    7    2    2   23]
 [   4    0    1   12    3  844   14    0   12    2]
 [   6    4    0    0    1    1  945    0    1    0]
 [   2   11    5    0    2    0    1  994    0   13]
 [  11    2    2    8    2    2    5    3  934    5]
 [   8    5    2    6    7    2    1   11    2  965]]


In [61]:
from sklearn.metrics import accuracy_score, classification_report

results['ACCURACY_SCORE'] = accuracy_score(y_test, y_pred)
results['DICT_REPORT'] = str(classification_report(y_test, y_pred, output_dict=True))
# results['DICT_REPORT'] = ''

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9683
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       980
           1       0.98      0.99      0.98      1135
           2       0.98      0.96      0.97      1032
           3       0.97      0.95      0.96      1010
           4       0.98      0.96      0.97       982
           5       0.98      0.95      0.96       892
           6       0.97      0.99      0.98       958
           7       0.97      0.97      0.97      1028
           8       0.96      0.96      0.96       974
           9       0.95      0.96      0.95      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000



In [13]:
prot_row(results)

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/M-Net_Protocol.xlsx'

In [63]:
pd.DataFrame([results])

,TIMESTAMP,NUMBER_OF_HIDDEN,ACCURACY_SCORE,MAE_1ST_FL,MSE_1ST_FL,R2_1ST_FL,MAE_1ST_BA,MSE_1ST_BA,R2_1ST_BA,MAE_2ND_FL,MSE_2ND_FL,R2_2ND_FL,MAE_2ND_BA,MSE_2ND_BA,R2_2ND_BA,MAE_TEST_BA,MSE_TEST_BA,R2_TEST_BA,DICT_REPORT,MSG
0,2020-12-26 21:42:11.050776,67,0.9683,0.437,0.593,-889.527,0.392,0.285,0.485,0.184,0.113,0.765,0.233,0.121,0.832,0.23,0.117,0.838,"{'0': {'precision': 0.9549019607843138, 'recal...",M-Net on python


In [64]:
results

{'ACCURACY_SCORE': 0.9683,
 'DICT_REPORT': "{'0': {'precision': 0.9549019607843138, 'recall': 0.9938775510204082, 'f1-score': 0.974, 'support': 980}, '1': {'precision': 0.9757785467128027, 'recall': 0.9938325991189427, 'f1-score': 0.9847228284591881, 'support': 1135}, '2': {'precision': 0.9783464566929134, 'recall': 0.9631782945736435, 'f1-score': 0.9707031250000001, 'support': 1032}, '3': {'precision': 0.9707955689828801, 'recall': 0.9544554455445544, 'f1-score': 0.9625561657513728, 'support': 1010}, '4': {'precision': 0.9802083333333333, 'recall': 0.9582484725050916, 'f1-score': 0.9691040164778579, 'support': 982}, '5': {'precision': 0.9813953488372092, 'recall': 0.9461883408071748, 'f1-score': 0.9634703196347031, 'support': 892}, '6': {'precision': 0.9662576687116564, 'recall': 0.9864300626304802, 'f1-score': 0.9762396694214875, 'support': 958}, '7': {'precision': 0.9650485436893204, 'recall': 0.9669260700389105, 'f1-score': 0.9659863945578231, 'support': 1028}, '8': {'precision': 0